# Analysing ACLED exported files

In [1]:
from arcgis.geometry import Geometry, Point
from arcgis.geometry.functions import buffer, relation
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

In [5]:
file_path = "D:/Intelligence/ACLED/2019-01-01-2019-12-31.csv"

In [6]:
def read_acled(file_path):
    return pd.read_csv(file_path)

def get_highest_spatial_precision(acled_data):
    return acled_data[acled_data["geo_precision"]==1]

def get_highest_temporal_precision(acled_data):
    return acled_data[acled_data["time_precision"]==1]

def get_protests(acled_data):
    return acled_data[acled_data["event_type"]=="Protests"]

def get_peaceful_protests(acled_data):
    acled_protests = get_protests(acled_data)
    return acled_protests[acled_protests["sub_event_type"]=="Peaceful protest"]

def get_most_precise_peaceful_protests(acled_data):
    return get_peaceful_protests(get_highest_spatial_precision(get_highest_temporal_precision(acled_data)))

def list_precisest_locations(acled_data):
    return get_highest_spatial_precision(acled_data)["location"].unique()

def count_precisest_locations(acled_data):
    return get_highest_spatial_precision(acled_data)["location"].value_counts()

In [7]:
acled_data = read_acled(file_path)

In [8]:
acled_data.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [9]:
acled_data["event_type"].value_counts()

Protests                      82867
Battles                       45367
Explosions/Remote violence    36451
Violence against civilians    23776
Riots                         15797
Strategic developments         9452
Name: event_type, dtype: int64

In [10]:
count_precisest_locations(get_peaceful_protests(acled_data))

Seoul City-Jongno              842
Jammu                          558
Ciudad de Mexico-Cuauhtemoc    515
Hyderabad                      445
Seoul City-Yeongdeungpo        393
                              ... 
Donga                            1
Guaruja                          1
Deryneia                         1
Mzimba                           1
Abbasabad                        1
Name: location, Length: 12195, dtype: int64

In [11]:
# Connect to ArcGIS Online anonymously
gis = GIS() 

def get_unique_locations(acled_data):
    acled_locations = {}
    for acled_index in acled_data.index:
        acled_location_name = acled_data["location"][acled_index]
        if (acled_location_name in acled_locations):
            acled_locations[acled_location_name]["hit_count"] += 1
        else:
            acled_point = Point({"x" : acled_data["longitude"][acled_index], "y" : acled_data["latitude"][acled_index], "spatialReference" : {"wkid" : 4326}})
            acled_locations[acled_location_name] = { "point": acled_point, "hit_count": 1 }
    
    return acled_locations

def to_spatial_dataframe(acled_data):
    acled_locations = {}
    acled_names = []
    acled_x = []
    acled_y = []
    acled_hit_counts = []
    acled_record_index = 0
    for acled_index in acled_data.index:
        acled_location_name = acled_data["location"][acled_index]
        if (acled_location_name in acled_locations):
            # Get the corresponding acled_record_index and update the hit_count
            record_index = acled_locations[acled_location_name]["record_index"]
            acled_hit_counts[record_index] += 1
        else:
            # Set the current acled_record_index for updating the corresponding hit_count
            acled_locations[acled_location_name] = { "record_index": acled_record_index}
            acled_names.append(acled_location_name)
            acled_x.append(acled_data["longitude"][acled_index])
            acled_y.append(acled_data["latitude"][acled_index])
            acled_hit_counts.append(1)
            acled_record_index += 1
    
    acled_dictionary = { "name": acled_names, "x": acled_x, "y": acled_y, "hit_count": acled_hit_counts }
    acled_data_frame = pd.DataFrame.from_dict(acled_dictionary)
    WGS84 = 4326
    return GeoAccessor.from_xy(acled_data_frame, x_column="x", y_column="y", sr=WGS84)

def get_europe_map():
    europe_map = gis.map("Europe")
    europe_map.basemap = "dark-gray-vector"
    return europe_map

In [14]:
#get_unique_locations(get_highest_spatial_precision(get_peaceful_protests(acled_data)))
acled_events = to_spatial_dataframe(get_most_precise_peaceful_protests(acled_data))
acled_events

,name,x,y,hit_count,SHAPE
0,Bhubaneswar,85.8338,20.2724,99,"{""x"": 85.8338, ""y"": 20.2724, ""spatialReference..."
1,Mahoba,79.8753,25.2905,2,"{""x"": 79.8753, ""y"": 25.2905, ""spatialReference..."
2,Wanju,127.2171,35.9220,2,"{""x"": 127.2171, ""y"": 35.922, ""spatialReference..."
3,Jalandhar,75.5791,31.3255,117,"{""x"": 75.5791, ""y"": 31.3255, ""spatialReference..."
4,Changwon,128.6056,35.1741,235,"{""x"": 128.6056, ""y"": 35.1741, ""spatialReferenc..."
...,...,...,...,...,...
11678,Gund Panah,74.6267,33.9565,1,"{""x"": 74.6267, ""y"": 33.9565, ""spatialReference..."
11679,Chertu,74.5800,33.9014,1,"{""x"": 74.58, ""y"": 33.9014, ""spatialReference"":..."
11680,Khari Karmara,74.0615,33.7066,1,"{""x"": 74.0615, ""y"": 33.7066, ""spatialReference..."
11681,Taichung City-Wucuo,120.5988,24.2754,1,"{""x"": 120.5988, ""y"": 24.2754, ""spatialReferenc..."


In [15]:
europe_map = get_europe_map()
acled_events.head().copy().spatial.plot(map_widget=europe_map, renderer_type="s")
europe_map

MapView(layout=Layout(height='400px', width='100%'))

In [68]:
import struct
float_bytes = struct.pack("f", 1.23)
struct.unpack("f", float_bytes)

(1.2300000190734863,)